Comparing hydraulic conductivity values using K0 + L + Rosetta 1 parameters with values obtained using K0 + L=0.5 + Rosetta 3 parameters

In [49]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MABarbadillo/
%cd 'GWP/GWP files'
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/MABarbadillo
/content/drive/MyDrive/MABarbadillo/GWP/GWP files


In [50]:
##Install rosetta

!pip install rosetta-soil
from rosetta import rosetta, SoilData

In [ ]:
####Version_1.1
####Using matric potential data from Mesotoobox - MesoSoilv1_3 Database / 'longtermmeans.csv'


import numpy as np   #Library that helps in numerical calculations
import pandas as pd  #Library used for handling structured data
                     #In this case, pandas read the csv file and convert the data into data frame (i.e., tables with rows and columns)

# Read the 'MesoSoilv1_3.csv' file, skipping the first row and using the second row as header
df = pd.read_csv('/content/drive/MyDrive/MABarbadillo/GWP/GWP files/MesoSoilv1_3.csv', header=0, skiprows=1)

# Extract the data from the specified columns
# In python, rows and columns start at 0
# .iloc is used for integer-location based indexing
# .values.tolist() converts the selected portion of the data frame into nested list format which is more flexible and covenient
# From the data frame, values needed are from depths 5, 25, and 60 which are all in odd rows
# 1::2 specifies to start reading from the second row and select the second row after that

data = df.iloc[1::2, [2, 3, 4, 5, 6, 7]].values.tolist()      #integers 2, 3, 4, 5, 6, and 7 specify the columns for %sand, silt, clay, bulk density (g/cm3), and volumetric water contents (cm3/cm3) at -33 and -1500 kPa, respectively
K0 = df.iloc[1::2, [13]].values.tolist()                      #K0 parameter from the csv file
K0 = [float(item) for sublist in K0 for item in sublist]      #converts the data to a float
L1 = df.iloc[1::2, [14]].values.tolist()                      #L parameter from the csv file
L1 = [float(item) for sublist in L1 for item in sublist]
L3 = 0.5   #default value

# Call the function to get hydraulic parameters
mean_1, stdev_1, codes_1 = rosetta(1, SoilData.from_array(data))    #for Rosetta Ver.1
mean_3, stdev_3, codes_3 = rosetta(3, SoilData.from_array(data))    #for Rosetta Ver.3

# Extract the hydraulic parameter values from the array using Rosetta Ver.1
# Converts everything into a float before performing the operation for m1 and m3
# item[] indicates the column number
theta_r1 = [float(item[0]) for item in mean_1]
theta_s1 = [float(item[1]) for item in mean_1]
alpha1 = [float(item[2]) for item in mean_1]
n1 = [float(item[3]) for item in mean_1]
m1 = [1 - (1 / value) for value in n1]

# Extract the hydraulic parameter values from the array using Rosetta Ver.3
theta_r3 = [float(item[0]) for item in mean_3]
theta_s3 = [float(item[1]) for item in mean_3]
alpha3 = [float(item[2]) for item in mean_3]
n3 = [float(item[3]) for item in mean_3]
m3 = [1 - (1 / value) for value in n3]

# Reading the 'longtermmeans.csv' file containing the matric potentials at 5, 25, 60 cm depth for 125 Mesonet sites
df1 = pd.read_csv('/content/drive/MyDrive/MABarbadillo/GWP/GWP files/longtermmeans.csv')
# Melt the DataFrame to convert columns MP05, MP25, and MP60 into rows
melted_df = df1.melt(id_vars=['STID'], value_vars=['MP05', 'MP25', 'MP60'], var_name='Depth', value_name='Matric_Potential')
# Sort the DataFrame by 'STID' and 'Depth' to maintain the order of the sites and depths
melted_df = melted_df.sort_values(by=['STID', 'Depth'])
# Reset the index
melted_df.reset_index(drop=True, inplace=True)
# Remove the first three entries (string variables)
melted_df = melted_df.iloc[3:]
# Convert the values in the 'Matric_Potential' column of the DataFrame melted_df into numeric data types and store inside matric_potential
matric_potential = melted_df['Matric_Potential'].apply(pd.to_numeric, errors='coerce').tolist()

# Convert matric potential to volumetric water content (vwc) for Rosetta Ver.1
# np.real extracts the real number
vwc1 = [theta_r1[i] + np.real((theta_s1[i] - theta_r1[i]) * (1 / (1 + (-alpha1[i] * matric_potential[i])**n1[i])**m1[i])) for i in range(len(matric_potential))]   #element-wise based calculation
# Convert matric potential to volumetric water content (vwc) for Rosetta Ver.3
vwc3 = [theta_r3[i] + np.real((theta_s3[i] - theta_r3[i]) * (1 / (1 + (-alpha3[i] * matric_potential[i])**n3[i])**m3[i]))for i in range(len(matric_potential))]    #cm3/cm3

# Convert volumetric water content to Effective saturation (Se) for Rosetta Ver.1
Se1 = [(vwc1[i] - theta_r1[i])/(theta_s1[i] - theta_r1[i]) for i in range(len(vwc1)) ]    #cm3/cm3
# Convert volumetric water content to Effective saturation (Se) for Rosetta Ver.3
Se3 = [(vwc3[i] - theta_r3[i])/(theta_s3[i] - theta_r3[i]) for i in range(len(vwc3)) ]    #cm3/cm3

# Calculate Hydraulic conductivity for Rosetta Ver.1
#K_Se1 = [(K0[i]*Se1[i]**L1[i]) * (1 - (1-Se1[i]**(n1[i]/n1[i]-1))**m1[i])**2 for i in range(len(Se1))]   #cm/day
# Calculate Hydraulic conductivity for Rosetta Ver.3
#K_Se3 = [(K0[i]*Se3[i]**L3) * (1 - (1-Se3[i]**(n3[i]/n3[i]-1))**m3[i])**2 for i in range(len(Se3))]   #cm/day


In [ ]:
#####Version_1.2
#####Using data directly from Mesonet for years 1998 to 2014

import numpy as np   #Library that helps in numerical calculations
import pandas as pd  #Library used for handling structured data
                     #In this case, pandas read the csv file and convert the data into data frame (i.e., tables with rows and columns)

# Read the 'MesoSoilv1_3.csv' file, skipping the first row and using the second row as header
# In python, rows and columns start at 0
# .iloc is used for integer-location based indexing
# .values.tolist() converts the selected portion of the data frame into nested list format which is more flexible and covenient

df = pd.read_csv('/content/drive/MyDrive/MABarbadillo/GWP/GWP files/MesoSoilv1_3.csv', header=0, skiprows=1)
data = df.iloc[1:, [2, 3, 4, 5, 6, 7]].values.tolist()      #integers 2, 3, 4, 5, 6, and 7 specify the columns for %sand, silt, clay, bulk density (g/cm3), and volumetric water contents (cm3/cm3) at -33 and -1500 kPa, respectively
K0 = df.iloc[1:, [13]].values.tolist()                      #K0 parameter from the csv file
L1 = df.iloc[1:, [14]].values.tolist()                      #L parameter from the csv file
L3 = 0.5   #default value

# Call the function to get hydraulic parameters
mean_1, stdev_1, codes_1 = rosetta(1, SoilData.from_array(data))    #for Rosetta Ver.1
mean_3, stdev_3, codes_3 = rosetta(3, SoilData.from_array(data))    #for Rosetta Ver.3

# Extract the hydraulic parameter values from the array using Rosetta Ver.1
# Converts everything into a float before performing the operation for m1 and m3
# item[] indicates the column number
theta_r1 = [float(item[0]) for item in mean_1]
theta_s1 = [float(item[1]) for item in mean_1]
alpha1 = [float(item[2]) for item in mean_1]
n1 = [float(item[3]) for item in mean_1]
m1 = [1 - (1 / value) for value in n1]

# Extract the hydraulic parameter values from the array using Rosetta Ver.3
theta_r3 = [float(item[0]) for item in mean_3]
theta_s3 = [float(item[1]) for item in mean_3]
alpha3 = [float(item[2]) for item in mean_3]
n3 = [float(item[3]) for item in mean_3]
m3 = [1 - (1 / value) for value in n3]

# To get the matric potential from #__# Mesonet sites for years 1998-2014 at depths 5, 25, 60, and 75 cm
# Read .csv file
# Daily reference temperature
####### Not sure about the data set: whether it matches the previous data or not
df1 = pd.read_csv('/content/drive/MyDrive/MABarbadillo/GWP/GWP files/Mesonet_file.csv')
ref_Temperature = df1.iloc[1:, [5, 6, 7, 8]].values.tolist()      #Reference temperature at depths 5, 25, 60, and 75 cm
df1.iloc[1:, [5, 6, 7, 8]] = df1.iloc[1:, [5, 6, 7, 8]].applymap(lambda x: np.nan if x < 0 else x)   #Replaces values <0 to NAN
c = 0.717  #calibration constant [kPa]
a = 1.788  #calibration constant [1/C]

matric_potential = []                                             #stores the calculated matric potential values for each depth
for temperatures in ref_Temperature:
    mp_depth = [-c * np.exp(a * temp) for temp in temperatures]   #stores matric potential values for each depth
    matric_potential.append(mp_depth)                             #ensures that matric potentials are calculated based on reference temperature values at specific depth

# Convert matric potential to volumetric water content (vwc) for Rosetta Ver.1
vwc1 = theta_r1 + (theta_s1 - theta_r1) * (1 / (1 + (-alpha1 * matric_potential)**n1)**m1)    #cm3/cm3
# Convert matric potential to volumetric water content (vwc) for Rosetta Ver.3
vwc3 = theta_r3 + (theta_s3 - theta_r3) * (1 / (1 + (-alpha3 * matric_potential)**n3)**m3)    #cm3/cm3

# Convert volumetric water content to Effective saturation (Se) for Rosetta Ver.1
Se1 = (vwc1 - theta_r1)/(theta_s1 - theta_r1)    #cm3/cm3
# Convert volumetric water content to Effective saturation (Se) for Rosetta Ver.3
Se3 = (vwc3 - theta_r3)/(theta_s3 - theta_r3)    #cm3/cm3

# Calculate Hydraulic conductivity for Rosetta Ver.1
K_Se1 = (K0*Se1**L1) * (1 - (1-Se1**(n1/n1-1))**m1)**2    #cm/day
# Calculate Hydraulic conductivity for Rosetta Ver.3
# K0 values from 'MesoSoilv1_3.csv' were used
# L3 = 0.5
K_Se3 = (K0*Se3**L3) * (1 - (1-Se3**(n3/n3-1))**m3)**2    #cm/day